<a href="https://colab.research.google.com/github/ssm951/chinese-genealogy/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCR

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/chinese-genealogy/

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 104.0 MB/s eta 0:00:00


In [3]:
!pip install --upgrade google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.5/467.5 kB 10.0 MB/s eta 0:00:00


In [4]:
from google.colab import files
from google.cloud import vision
import fitz
from PIL import Image
import cv2
import io
import os
# only need to run once
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


Saving arcane-atom-206408-57eab0d8fd7e.json to arcane-atom-206408-57eab0d8fd7e.json
User uploaded file "arcane-atom-206408-57eab0d8fd7e.json" with length 2362 bytes


In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = fn

In [39]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/chinese-genealogy/data/'
#sourcename = 'Hong_Kong_Mo_genealogy_1999'
#sourcename = 'Genealogy Book Deng Family Anrenli Village - Family Edition _ 安仁里鄧氏族譜 - 家庭版本 _ 安仁里邓氏族谱 - 家庭版本'
sourcename = 'Genealogy Book Deng Family Anrenli Village - Village Edition _ 安仁里鄧氏族譜 - 村版本 _ 安仁里邓氏族谱 - 村版本'
pdf_path = folder_path + sourcename + '/' + sourcename + '.pdf'

def convert_pdf_to_images(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page in pdf_document:
      pix = page.get_pixmap()
      image = Image.open(io.BytesIO(pix.tobytes("png")))
      #image = image.rotate(-90, expand=True)
      images.append(image)
    return images

images = convert_pdf_to_images(pdf_path)

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/chinese-genealogy/data/'
sourcename = '吳氏族譜溯源記'
images_path = folder_path + sourcename + '/raw/'
images = []
for fn in sorted(os.listdir(images_path)):
  image = Image.open(images_path + fn)
  images.append(image)

In [16]:
def detect_text_google_vision(image):
    client = vision.ImageAnnotatorClient()
    image_byte_array = io.BytesIO()
    image.save(image_byte_array, format='PNG')
    image_content = image_byte_array.getvalue()
    image = vision.Image(content=image_content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    if texts:
        return texts[0].description.replace(' ', '')
    return ''

texts_google = []
for i, image in enumerate(images):
    text = detect_text_google_vision(image)
    texts_google.append(text)

In [17]:
average = sum([len(text.split('\n')) for text in texts_google]) / len(texts_google)
processed_text = ""
for text in texts_google:
  lines = text.split('\n')
  for line in lines:
    line = line.strip()
    if len(line) == 0:
      continue
    elif all(char in '，、︒。°！？：；,.-1234567890 \t\n' for char in line):
      continue
    elif len(line) > 0.8 * average:
      processed_text += line
    else:
      processed_text += line + "\n"

In [10]:
processed_text = processed_text.replace('。', '。\n')
processed_text = processed_text.replace('︒', '︒\n')
processed_text = processed_text.replace('！', '！\n')
processed_text = processed_text.replace('？', '？\n')
processed_text = processed_text.replace('……', '……\n')
processed_text = processed_text.replace('；', '；\n')

In [18]:
print(processed_text)

供王墓祭祀
冬
宋仁宗大聖中七十七代孫吳感南增置田二百五十畝於墓南破山港側今灑掃曹伯七等耕種人仲雍墓詩
仲雍古墓旣邱墟塚上閑雲日卷舒奠寫不聞陳簠簋夷端合杷樵蘇杜鹃啼血春風老狡兎穿瑩深夜孤遇客無勞增悵惘至今讓德重勾吳
季札墓在常州晋陵縣北七十里申浦之西於江陰陽鄉驕陽門外三十里申港之側墓高一丈八尺域四十三步孔子題延陵吳季子之墓日
מתת
一百〇九
一百一十
前人季子墓古稱尼父篆始沒春秋義十字固莫測磨敲任牧童侵剝因野蘚嗟爾後之人萬言書不淺晋明帝太甯元年勅祀爲王禮建殿於墓南三十步周垣廣三百六十五步復命五十戶守衛王墓宋武帝御贊亭在殿前一十五步
唐太宗貞觀十三年勅重大門境賜金爐花瓶祭器一壇建庫在殿旁收貯
唐開元三十五年帝命殷仲容摹刻孔子書十字碑傳世其大徑尺體勢奇偉
季子掛劍處
羅隱
忠貞者必信所信交道深賢哉吳季子可稱莫逆心吹毛霜刀過千金生許徐君死掛林寶劍徒稱無價寶心罐
行更貴不欺心
詠季子掛劍曲
李東陽風
長劍許烈士寸心報知己死者豈知我心元不死平生讓國心耿耿方知此
歷朝祀廟祀墓儀文實繁不能殫姑擧大概味宋武帝賛文
古吳國君泰伯神位像正南向坐
宋武帝御製三讓王贊宋永初元年三月初五日之寶維王三讓遂成文武古公廼親后稷其祖立國東吳載論中魯日月光華乾坤心腑後嗣彌昌慶流宏博我敬贊辭用昭千古
一百十一
一百十二
古吳國君仲雍神位像東片上南坐
宋武帝御製恭孝王贊宋永初五年三月初五日之寶維王恭孝繼以三辭協承先志丕閘周基無思無爲不識不知清而且廉權而適宜心同道合德與仁齊流風餘韻萬世永思
古吳延陵季子神位𩵚黼銼繊訨粢九族兗九章執圭垂綉裳赤寫
以上俱冕九
帝御製延陵王贊宋永初元年三月初五日之寶維王延陵全義讓國見禮之政聞樂之德觀風審音掛劍酹心懷哉高風無古無全
上三公事蹟碑於廟石於墓採摭鑑史雜錄悉屬確據不可誣也A
會
季札公雖屬旁支爲有周一代大賢孔聖表爲君子猶極深景仰誌之亦足以焕吳氏之光FamilySear
一百十三
吳氏先始世系
一百十四
后稷公銘
名棄生於顓頊七十年丙子歲卒於帝堯十八年癸亥享一百六十歲葬於納姞氏傳染璽吳楘璽公繼后稷傳叔望
吳叔望公關樂園糠杯雞(花史記)(周本記)明氏自后稷後不窟
【節皇僕差弗毀隃公非高圍亞圉公叔祖類古公亶父凡十三世合王季文王卽國王公
卽劉
國慶
不窟公
語所云十五王也組紺卽公叔祖類乃太王之父也叔望二代名號史記未有載云難以確實傳至古公亶父以上所有疑問俱按中記補以備考矣A傳鞠陶

In [19]:
#os.mkdir(folder_path + sourcename)
with open(folder_path + sourcename + '/ocr_processed.txt', 'w') as f:
  f.write(processed_text)